<a href="https://colab.research.google.com/github/danykoud/Test-/blob/main/NLP_Problem.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import pandas as pd
import httpimport

with httpimport.remote_repo(['lm_helper'], 'https://raw.githubusercontent.com/jasoriya/CS6120-PS2-support/master/utils/'):
  from lm_helper import get_train_data, get_test_data

In [ ]:
#  get the train and test data
train = get_train_data()
test, test_files = get_test_data()


TASK - 1 = 10 points :
##**DESCRIPTION** :
Collect statistics on the unigram, bigram, and trigram character counts.
Statistics such as:
a)Length of unigram vocabulary
b)Count of total unigrams
c)Length of bigram vocab
d)Count of total bigrams
e)Length of trigram vocabulary
f)Count of total trigrams

In [ ]:
from sklearn.feature_extraction.text import CountVectorizer
import numpy as np

# formatting the doc to be used for training

tList = []
for each in train:
    for l in each:
        tList.append(" ".join(l))
docList= tList[:8000]
def getUnigramVals(docList):
    '''

    Generate a dictionary with unigrams as keys and their counts for given input as values
    e.g: {'a':53,'b':65}
    
    '''
    unigram_vectorizer = CountVectorizer() #use this variable for defining CountVectorizer()
    unigram_vector = unigram_vectorizer.fit_transform(docList) #use this variable for fit transform
    unigram_counts =unigram_vector.toarray().sum(axis=0)
     #use this counting unigrams
    return dict(zip(unigram_vectorizer.get_feature_names(),unigram_counts))


def getBigramVals(docList):
    '''

    Generate a dictionary with unigrams as keys and their counts for given input as values
    e.g: {'ab':53,'bg':65}
    
    '''
    bigram_vectorizer = CountVectorizer(ngram_range=(2, 2))
    bigram_vector =bigram_vectorizer.fit_transform(docList)
    bigram_counts =bigram_vector.toarray().sum(axis=0)
    return dict(zip(bigram_vectorizer.get_feature_names(), bigram_counts))


def getTrigramVals(docList):
    '''

    Generate a dictionary with unigrams as keys and their counts for given input as values
    e.g: {'abd':53,'bgv':65}
    
    '''
    trigram_vectorizer =  CountVectorizer(ngram_range=(3, 3))
    trigram_vector = trigram_vectorizer.fit_transform(docList)
    trigram_counts = trigram_vector.toarray().sum(axis=0)
    return dict(zip((trigram_vectorizer.get_feature_names()), (trigram_counts)))

# Get the respective uni,i and trigram values for the entire training data.
uniGram = getUnigramVals(docList)
biGram = getBigramVals(docList)
triGram = getTrigramVals(docList)
unigram_sum = sum(uniGram.values())

# Print the values of uni, tri and bigrams.
print("Length of unigram vocab:" + str(len(uniGram)))
print("Count of total unigrams:" + str(sum(uniGram.values())))
print("Length of bigram vocab:" + str(len(biGram)))
print("Count of total bigrams:" + str(sum(biGram.values())))
print("Length of trigram vocab:" + str(len(triGram)))
print("Count of total trigrams:" + str(sum(triGram.values())))


TASK - 2 = 15 points:
##**DESCRIPTION** :
Calculate the perplexity for each document in the test set using the linear interpolation smoothing method. For determining λs for linear interpolation of the trigram, bigram, and unigram models, you can divide the training data into a new training set (80%) and a held-out set (20%). Then choose ~10 random pairs of  (𝜆3,𝜆2)  such that  1>𝜆3>𝜆2>0  and  ∑3𝑖=1𝜆𝑖=1  to test on held-out data.

Some documents in the test set are in Brazilian Portuguese. Identify them as follows:

Sort by perplexity and set a cut-off threshold. All the documents above this threshold score should be categorized as Brazilian Portuguese.

Print the file names (from test_files) and perplexities of the documents above the threshold

    file name, score
    file name, score
    . . .
    file name, score
Copy this list of filenames and manually annotate them as being correctly or incorrectly labeled as Portuguese.

In [ ]:
# Your code here

from sklearn.model_selection import train_test_split
import numpy as np
import random
import nltk
# split the training data into 80-20 ratio for finding lambdas  
docList_train, docList_test = train_test_split( docList, test_size=0.20, random_state=42) # use train_test_split on docList 
                               
 
# Respective uni, bi and trigrams for the 80 splitted training data
unigram_trained = getUnigramVals(docList_train)
bigram_trained = getBigramVals(docList_train)
trigram_trained = getTrigramVals(docList_train)
unigram_trained_sum = sum(unigram_trained.values())
bigram_trained_sum = sum(bigram_trained.values())
trigram_trained_sum = sum(trigram_trained.values())

# Uni, bi and trigram probability computation methods. 
def getUnigramProbabilitty(l1, uniVals, unigramSum):
    
    if l1 not in uniVals:
      uniVals[l1]=1
      # return 0
    return uniVals[l1]/unigramSum

def getBigramProbability(l1, l2, uniVals, biVals):
    pair = l1,l2
    pair=" ".join(pair)
    if pair not in biVals:
      biVals[pair]=1
    if l1 not in uniVals:
      uniVals[l1]=1
    # if pair not in biVals or l1 not in uniVals:
    #   return 0
    return biVals[pair]/uniVals[l1]
    
def getTrigramProbability(l1, l2, l3, triVals, biVals):
    tr= l1,l2,l3
    pair= l1,l2
    pair=" ".join(pair)
    tr=" ".join(tr) 
    if tr not in triVals:
      triVals[tr]=1
    if pair not in biVals:
      biVals[pair]= 1
    # if tr not in triVals or pair not in biVals:
    #   return 0
    return triVals[tr]/biVals[pair]
# compute the perplexity with linear interpolation   
def computePerplexityLinearInterpolation(doc, lambda1, lambda2, lambda3, uniVals, biVals, triVals, uniSum):
    entropy = 0
    for i in range(0, len(doc)-2):
        l1 = doc[i]
        l2 = doc[i+1]
        l3 = doc[i+2]
        p3=getTrigramProbability(l1, l2, l3, triVals, biVals)
        p2=getBigramProbability(l1, l2, uniVals, biVals)
        p1=getUnigramProbabilitty(l1, uniVals, uniSum)
        probability = lambda1*p3 + lambda2*p2 + lambda3*p1
        entropy += (- probability * np.log2(probability))
    perplexity = 2** entropy
    return perplexity

# Make the held-out documents i.e books into a single stream or string of data as per the process followed in eisenstein for perplexity computation. Page: 139
held_out_doc = " ".join(docList_test)
# held_out_doc = docList_test.strip().split()                            #nltk.word_tokenize(docList_test)

lambdasToTry =list() #  eg : [[0.1, 0.1, 0.8],[0.2,0.35,0.45],.........]]    
k = np.round(np.random.random(100),2)
for i in range(len(held_out_doc[:2000])):
    d1= random.choice(k)
    d2= random.choice(k)
    d3= random.choice(k)
    if d1+ d2+d3 ==1 and d1!= d2!=d3  :
      lambdasToTry.append([d1, d2,d3])
# list for held-out perplexities
perps = []
for a in lambdasToTry:
    lambda1, lambda2, lambda3 = a[0], a[1], a[2]
    perps.append(computePerplexityLinearInterpolation(held_out_doc, lambda1, lambda2, lambda3, unigram_trained, bigram_trained, trigram_trained,unigram_trained_sum))

print(perps)

# final lambdas based on minimum perplexities on held-out data
finalLambdas = min(perps)
print(finalLambdas)
 

In [ ]:
testDocList = []
#create the test data list in structured format
docList_test =" ".join(docList_test)
docList_test = docList_test.strip().split()
for test in docList_test:
  testDocList.append(test)

#list for storing perplexities of test docs
perplexities_test_docs_interpolation = []
for word in testDocList:
  perplexities_test_docs_interpolation.append(computePerplexityLinearInterpolation(word, lambda1, lambda2, lambda3, unigram_trained, bigram_trained, trigram_trained,unigram_trained_sum))
print(perplexities_test_docs_interpolation)


In [ ]:
#sort test docs based on perplexity on perplexities_test_docs_interpolation
testDocperps = {}
for word in docList_test:
  for perp in perplexities_test_docs_interpolation:
    testDocperps[word]= float(perp)
sorted(testDocperps)
#print all perplexities of docs in sorted order to find threshold
print(testDocperps)


#set the perplexity threshold based on observation after sorted values are printed.
perplexity_threshold_for_interpolation = None

In [ ]:
#Classify non english docs based on perplexity threshold and printing their filenames along with perplexity for manual validation on perplexities_test_docs_interpolation

...........

In [ ]:
#print few lines from the doc, perplexity value, filename and classification by the model by using perplexities_test_docs_interpolation
..........

**TASK** - 3 = 10 points :
##**DESCRIPTION** :
Build a trigram language model with add-λ smoothing (use λ = 0.1).

Sort the test documents by perplexity and perform a check for Brazilian Portuguese documents as above:

Observe the perplexity scores and set a cut-off threshold. All the documents above this threshold score should be categorized as Brazilian Portuguese.

Print the file names and perplexities of the documents above the threshold

  file name, score
  file name, score
  . . .
  file name, score
Copy this list of filenames and manually annotate them for correctness.

In [ ]:
# Method for calculating trigram probability 
def getTrigramProbabilityWithLambdaSmoothing(l1, l2, l3, triVals, biVals, lambdaVal): 
    tr= l1,l2,l3
    pair= l1,l2
    pair=" ".join(pair)
    tr=" ".join(tr) 
    if tr not in triVals:
      triVals[tr]=1
    if pair not in biVals:
      biVals[pair]= 1
    # if tr not in triVals or pair not in biVals:
    #   return 0
    return (triVals[tr]+ 1)/(biVals[pair]+ lambdaVal )
 
            

# Method for perplexity calculation with lambda smoothing
def computePerplexityWithLambdaSmoothing(doc, biVals, triVals):
    entropy = 0
    for i in range(len(doc)-2):
        l1 = doc[i]
        l2 = doc[i+1]
        l3 = doc[i+2]
        probability = getTrigramProbabilityWithLambdaSmoothing(l1, l2, l3, triVals, biVals, 0.1)
        entropy += (- np.log2(probability))
    perplexity = 2**entropy 
    return perplexity

#Testing lambda smoothened trigram perplexity on held-out data
#print(computePerplexity(held_out_doc, bigramValsTrain, bigramVocabTrain, trigramValsTrain, trigramVocabTrain))

# list for perplexities and details in test data
perplexities_test_docs_lambda_smoothing = []
for each in testDocList:
     perplexities_test_docs_lambda_smoothing.append(computePerplexityWithLambdaSmoothing(each, bigramValsTrain, bigramVocabTrain, trigramValsTrain, trigramVocabTrain))

In [ ]:
#sort perplexities_test_docs_lambda_smoothing based on perplexity 
.......

#print all the perplexities in sorted order to figure out threshold
.......

#perplexity threshold set based on observation after sorted values are printed.
perplexity_threshold_for_lambda_smoothing = #select value here

In [ ]:

#Classify non english docs based on perplexity threshold and print their filenames along with perplexity for manual validation on perplexities_test_docs_lambda_smoothing
.......

In [ ]:
#print few lines from the doc, perplexity value, filename and classification by the model perplexities_test_docs_lambda_smoothing.
...........

**TASK** - 4 = 5 points
Based on your observation from above questions, compare linear interpolation and add-λ smoothing by listing out their pros and cons.

**END OF PART-A**

**PART-B**

**OBJECTIVE** :

In this part, you will be implementing logistic regression for sentiment analysis on tweets.

Problem Statement :
Given a tweet, you will decide if it has a positive sentiment or a negative.

Specifically you will:
Learn how to extract features for logistic regression given some text
Implement logistic regression from scratch
Apply logistic regression on a natural language processing task
Test using your logistic regression
Predict your own tweet.
Note : 
Initial steps will be same as in Assignment-1. You will be working on a data set of tweets.
¶

In [ ]:
import re
import string
from os import getcwd

import pdb

import numpy as np
import pandas as pd

import nltk
from nltk.corpus import stopwords, twitter_samples
from nltk.stem import PorterStemmer
from nltk.tokenize import TweetTokenizer

from matplotlib.patches import Ellipse
import matplotlib.transforms as transforms

**Task 0.1 - Preprocessing the Tweets**: ( 2.5 points)

In this task you have to start with cleaning the tweet for which you will write a function named clean_tweet which will remove hyperlinks, noise, punctuations, hashtags, old style retweet text "RT", share market tickers like $FB, use stemming to only keep track of one variation of each word and in the end will tokenize them. Look for the hint in imports to tokenize the tweet. Then you will write a helper function for determining the word frequency after which you will split the data into training and validation set. While doing this do not make unnceseaary assumptions regarding the lengths of tweets.

In [ ]:
#Preprocessing tweets
def process_tweet(tweet):
    #Remove old style retweet text "RT"
    tweet2 = re.sub(r'^RT[\s]','', tweet)
    
    #Remove hyperlinks
    tweet2 = re.sub(r'https?:\/\/.*[\r\n]*','', tweet2)
    
    #Remove hastags
    #Only removing the hash # sign from the word
    tweet2 = re.sub(r'#','',tweet2)
        
    # instantiate tokenizer class
    tokenizer = TweetTokenizer(preserve_case=False,    strip_handles=True, reduce_len=True)
    
    # tokenize tweets
    tweet_tokens = tokenizer.tokenize(tweet2)    
        
    #Import the english stop words list from NLTK
    stopwords_english = stopwords.words('english') 
    
    #Creating a list of words without stopwords
    tweets_clean = []
    for word in tweet_tokens:
        if word not in stopwords_english and word not in string.punctuation:
            tweets_clean.append(word)
        
    #Instantiate stemming class
    stemmer = PorterStemmer()
    
    #Creating a list of stems of words in tweet
    tweets_stem = []
    for word in tweets_clean:
        stem_word = stemmer.stem(word)
        tweets_stem.append(stem_word)
        
    return tweets_stem

In [ ]:
# TASK 1 CELL

# Helper function to find word frequency
def find(frequency, word, label):
    '''
    Params:
        frequency: a dictionary with the frequency of each pair (or tuple)
        word: the word to look up
        label: the label corresponding to the word
    Return:
        n: the number of times the word with its corresponding label appears.
    '''
    n = 0  
    
    pair = (word, label)
    if pair in frequency:
        n = frequency[pair]
    return n

If you are running this notebook in your local computer, don't forget to download the twitter samples and stopwords from nltk.

In [ ]:
# Run this cell
nltk.download('stopwords')
nltk.download('twitter_samples')

In [ ]:
# add folder, tmp2, from our local workspace containing pre-downloaded corpora files to nltk's data path
filePath = f"{getcwd()}/../tmp2/"
nltk.data.path.append(filePath)

In [ ]:
# get the sets of positive and negative tweets
positive_tweets = twitter_samples.strings('positive_tweets.json')
all_negative_tweets = twitter_samples.strings('negative_tweets.json')

# split the data into train and validation set
test_positive = positive_tweets[4000:]
train_positive = positive_tweets[:4000]
test_negative = all_negative_tweets[4000:]
train_negative = all_negative_tweets[:4000]
train_x = train_positive + train_negative
test_x = test_positive + test_negative
train_y = np.append(np.ones((len(train_positive), 1)), np.zeros((len(train_negative), 1)), axis=0)
test_y = np.append(np.ones((len(test_positive), 1)), np.zeros((len(test_negative), 1)), axis=0)


In [ ]:
tweets= pd.DataFrame.from_dict(dict(tweet=train_x, label=train_y.ravel()))
tweets

In [ ]:
# Print the shape train and test sets
print("train_y.shape = " + str(train_y.shape))
print("test_y.shape = " + str(test_y.shape))

### **Task 0.2 - Tweet Counter: (2.5 points)** 


To help train your naive bayes model, you will need to build a dictionary where the keys are a (word, label) tuple and the values are the corresponding frequency.  Note that the labels we'll use here are 1 for positive and 0 for negative.

You will also implement a `find()` helper function that takes in the `freqs` dictionary, a word, and a label (1 or 0) and returns the number of times that word and label tuple appears in the collection of tweets.

For example: given a list of tweets `["i am rather excited", "you are rather happy"]` and the label 1, the function will return a dictionary that contains the following key-value pairs:

{
    ("rather", 1): 2
    ("happi", 1) : 1
    ("excit", 1) : 1
}

- Notice how for each word in the given string, the same label 1 is assigned to each word.
- Notice how the words "i" and "am" are not saved, since it was removed by clean_tweet because it is a stopword.
- Notice how the word "rather" appears twice in the list of tweets, and so its count value is 2.

#### **Instructions**
Create a function `tweet_counter()` that takes a list of tweets as input, cleans all of them, and returns a dictionary.
- The key in the dictionary is a tuple containing the stemmed word and its class label, e.g. ("happi",1).
- The value the number of times this word appears in the given collection of tweets (an integer).

In [ ]:
# TASK 2 CELL

def tweet_counter(output, tweets, tweet_senti):
    '''
    Params:
        output: a dictionary that will be used to map each pair to its frequency
        tweets: a list of tweets
        tweet_senti: a list corresponding to the sentiment of each tweet (either 0 or 1)
    Return:
        output: a dictionary mapping each pair to its frequency
    '''
    # method#1
    # for label, tweet in zip(tweet_senti, tweets):
    #   for word in process_tweet(tweet):
    #     pair = (word, label)
    #     output[pair] = output.get(pair, 0) + 1
    # method#2
    for label, tweet in zip(tweet_senti, tweets):
        for word in process_tweet(tweet):
            pair=(word,label)
            if pair in output:
                output[pair] += 1
            else:
              output[pair] = 1

    return output

frequency_dict= freqs= tweet_counter({}, train_x, train_y.ravel())

##**Logistic regression**
¶
Task-1 = 15 Points
Task 1.1: Sigmoid
The sigmoid function is defined as:

ℎ(𝑧)=1/1+exp−𝑧(1)

It maps the input 'z' to a value that ranges between 0 and 1, and so it can be treated as a probability.

In [ ]:
import math
def sigmoid(z): 
    '''
    Input:
        z: is the input (can be a scalar or an array)
    Output:
        h: the sigmoid of z
    '''
    
   
    # calculate the sigmoid of z
    h = 1 / (1 + np.exp(-z))
    
    
    return h

In [ ]:
# Test your sigmoid function by taking z=0 and z = 4.92. check and print if it works correctly as success.
if (sigmoid(0) == 0.5):
    print('SUCCESS!')
else:
    print('Oops!')

if (sigmoid(4.92) == 0.9927537604041685):
    print('CORRECT!')
else:
    print('Oops again!')

**Logistic regression**: 

regression and a sigmoid
Logistic regression takes a regular linear regression, and applies a sigmoid to the output of the linear regression.

Regression:

𝑧=𝜃0𝑥0+𝜃1𝑥1+𝜃2𝑥2+...𝜃𝑁𝑥𝑁
 
Note that the  𝜃  values are "weights". If you took the Deep Learning Specialization, we referred to the weights with the w vector. In this course, we're using a different variable  𝜃  to refer to the weights.

Logistic regression

ℎ(𝑧)=11+exp−𝑧
 
𝑧=𝜃0𝑥0+𝜃1𝑥1+𝜃2𝑥2+...𝜃𝑁𝑥𝑁
 
We will refer to 'z' as the 'logits'.

Task 1.2 Cost function and Gradient
The cost function used for logistic regression is the average of the log loss across all training examples:

𝐽(𝜃)=−1𝑚∑𝑖=1𝑚𝑦(𝑖)log(ℎ(𝑧(𝜃)(𝑖)))+(1−𝑦(𝑖))log(1−ℎ(𝑧(𝜃)(𝑖)))(5)

𝑚 is the number of training examples
𝑦(𝑖) is the actual label of the i-th training example.
ℎ(𝑧(𝜃)(𝑖)) is the model's prediction for the i-th training example.
The loss function for a single training example is

𝐿𝑜𝑠𝑠=−1×(𝑦(𝑖)log(ℎ(𝑧(𝜃)(𝑖)))+(1−𝑦(𝑖))log(1−ℎ(𝑧(𝜃)(𝑖))))

All the ℎ values are between 0 and 1, so the logs will be negative. That is the reason for the factor of -1 applied to the sum of the two loss terms.
Note that when the model predicts 1 (ℎ(𝑧(𝜃))=1) and the label 𝑦 is also 1, the loss for that training example is 0.
Similarly, when the model predicts 0 (ℎ(𝑧(𝜃))=0) and the actual label is also 0, the loss for that training example is 0.
However, when the model prediction is close to 1 (ℎ(𝑧(𝜃))=0.9999) and the label is 0, the second term of the log loss becomes a large negative number, which is then multiplied by the overall factor of -1 to convert it to a positive loss value. −1×(1−0)×𝑙𝑜𝑔(1−0.9999)≈9.2 The closer the model prediction gets to 1, the larger the loss.
Update the weights
To update your weight vector 𝜃, you will apply gradient descent to iteratively improve your model's predictions.
The gradient of the cost function 𝐽 with respect to one of the weights 𝜃𝑗 is:

∇𝜃𝑗𝐽(𝜃)=1𝑚∑𝑖=1𝑚(ℎ(𝑖)−𝑦(𝑖))𝑥𝑗(5)
'i' is the index across all 'm' training examples.

'j' is the index of the weight 𝜃𝑗, so 𝑥𝑗 is the feature associated with weight 𝜃𝑗
To update the weight 𝜃𝑗, we adjust it by subtracting a fraction of the gradient determined by 𝛼:

𝜃𝑗=𝜃𝑗−𝛼×∇𝜃𝑗𝐽(𝜃)

The learning rate 𝛼 is a value that we choose to control how big a single update will be.

Instructions: Implement gradient descent function
The number of iterations num_iters is the number of times that you'll use the entire training set.
For each iteration, you'll calculate the cost function using all training examples (there are m training examples), and for all features.
Instead of updating a single weight 𝜃𝑖 at a time, we can update all the weights in the column vector:

𝜃=𝜃0𝜃1𝜃2⋮𝜃𝑛

𝜃 has dimensions (n+1, 1), where 'n' is the number of features, and there is one more element for the bias term 𝜃0 (note that the corresponding feature value 𝐱0 is 1).

The 'logits', 'z', are calculated by multiplying the feature matrix 'x' with the weight vector 'theta'. 𝑧=𝐱𝜃

𝐱 has dimensions (m, n+1)

𝜃: has dimensions (n+1, 1)

𝐳: has dimensions (m, 1)

The prediction 'h', is calculated by applying the sigmoid to each element in 'z': ℎ(𝑧)=𝑠𝑖𝑔𝑚𝑜𝑖𝑑(𝑧), and has dimensions (m,1).

The cost function 𝐽 is calculated by taking the dot product of the vectors 'y' and 'log(h)'. Since both 'y' and 'h' are column vectors (m,1), transpose the vector to the left, so that matrix multiplication of a row vector with column vector performs the dot product.

𝐽=−1𝑚×(𝐲𝑇⋅𝑙𝑜𝑔(𝐡)+(1−𝐲)𝑇⋅𝑙𝑜𝑔(1−𝐡))

The update of theta is also vectorized. Because the dimensions of 𝐱 are (m, n+1), and both 𝐡 and 𝐲 are (m, 1), we need to transpose the 𝐱 and place it on the left in order to perform matrix multiplication, which then yields the (n+1, 1) answer we need:

𝜃=𝜃−𝛼𝑚×(𝐱𝑇⋅(𝐡−𝐲))

In [ ]:
def gradientDescent(x, y, theta, alpha, num_iters):
    '''
    This function should return (J, theta) by taking (x, y, theta, alpha, num_iters) as inputs, where
        x: matrix of features which is (m,n+1)
        y: corresponding labels of the input matrix x, dimensions (m,1)
        theta: weight vector of dimension (n+1,1)
        alpha: learning rate
        num_iters: number of iterations you want to train your model for
        J: the final cost
        theta: your final weight vector
    '''
    
    # get 'm', the number of rows in matrix x
    m = x.shape[0]
    
    for i in range(0, num_iters):
        
        # get z, the dot product of x and theta
        z = np.dot(x, theta)
        
        # get the sigmoid of z
        h = sigmoid(z)
       
        
        # calculate the cost function
        J =-1./m * (np.dot(y.transpose(), np.log(h)) + np.dot((1-y).transpose(),np.log(1-h)))  

        # update the weights theta
        theta = theta - (alpha/m)*(np.dot(np.transpose(x),(h-y)))
        
    ### END CODE HERE ###
    J = float(J)
    return J, theta

In [ ]:
# expected output should match with your output
# Check the function
# Construct a synthetic test case using numpy PRNG functions
np.random.seed(1)
# X input is 10 x 3 with ones for the bias terms
tmp_X = np.append(np.ones((10, 1)), np.random.rand(10, 2) * 2000, axis=1)
# Y Labels are 10 x 1
tmp_Y = (np.random.rand(10, 1) > 0.35).astype(float)

# Apply gradient descent
# gradientDescent = np.vectorize(gradientDescent)
tmp_J, tmp_theta = gradientDescent(tmp_X, tmp_Y, np.zeros((3, 1)), 1e-8, 700)
print(f"The cost after training is {tmp_J:.8f}.")
print(f"The resulting vector of weights is {[round(t, 8) for t in np.squeeze(tmp_theta)]}")

In [ ]:
def extract_features(tweet, freqs):
    '''
    This function should take tweet, freqs as input and return x, where:
        tweet: a list of words for one tweet
        freqs: a dictionary corresponding to the frequencies of each tuple (word, label)
        x: a feature vector of dimension (1,3)
    '''
  
    # process_tweet tokenizes, stems, and removes stopwords
    word_l = process_tweet(tweet)
    
    # 3 elements in the form of a 1 x 3 vector
    x = np.zeros((1, 3)) 
    
    #bias term is set to 1
    x[0,0] = 1 
    
    
    # loop through each word in the list of words
    for word in word_l:
        
        # increment the word count for the positive label 1
        x[0,1] += freqs.get((word,1),0)
        
        # increment the word count for the negative label 0
        x[0,2] += freqs.get((word,0),0)
        
    assert(x.shape == (1, 3))
    return x

In [ ]:
# Check your function
# expected output should match with your output
# test1: on training data
tmp1 = extract_features(train_x[0], freqs)
print(tmp1)

In [ ]:
# test 2:
# expected output should match with your output
# check for when the words are not in the freqs dictionary
tmp2 = extract_features('blorb bleeeeb bloooob', freqs)
print(tmp2)

**Expected output**

[[1. 0. 0.]]

**Task 3: Training Your Model -** (5 Points)

To train the model:

Stack the features for all training examples into a matrix X.
Call gradientDescent, which you've implemented above.

In [ ]:
# collect the features 'x' and stack them into a matrix 'X'
# Expected output should map to your output
X = np.zeros((len(train_x), 3))
for i in range(len(train_x)):
    X[i, :]= extract_features(train_x[i], freqs)

# training labels corresponding to X
Y = train_y

# Apply gradient descent
J, theta = gradientDescent(X, Y, np.zeros((3, 1)), 1e-9, 1500) #call gradient descent by appropriate parameters.
print(f"The cost after training is {J:.8f}.")
print(f"The resulting vector of weights is {[round(t, 8) for t in np.squeeze(theta)]}")

**Expected Output:**

The cost after training is 0.24216529.
The resulting vector of weights is [7e-08, 0.0005239, -0.00055517]

**Task 4**: Test your logistic regression - (10 Points)
It is time for you to test your logistic regression function on some new input that your model has not seen before.

**Instructions:** 

Write predict_tweet
Predict whether a tweet is positive or negative.

Given a tweet, process it, then extract the features.
Apply the model's learned weights on the features to get the logits.
Apply the sigmoid to the logits to get the prediction (a value between 0 and 1).
𝑦𝑝𝑟𝑒𝑑=𝑠𝑖𝑔𝑚𝑜𝑖𝑑(𝐱⋅𝜃)

In [ ]:
def predict_tweet(tweet, freqs, theta):
    '''
    This function should take (tweet, freqs, theta) as input and return y_pred as output where
        tweet: a string
        freqs: a dictionary corresponding to the frequencies of each tuple (word, label)
        theta: (3,1) vector of weights
        y_pred: the probability of a tweet being positive or negative
    '''
    
    # extract the features of the tweet and store it into x
    x = extract_features(tweet, freqs)    # call extract features function with appropriate parameters 
    
    # make the prediction using x and theta by calling sigmoid function
    y_pred = sigmoid(np.dot(x,theta))
    
    return y_pred

In [ ]:
# Run this cell to test your function
# Expected output should match to your output.
for tweet in ['I am happy', 'I am bad', 'this movie should have been great.', 'great', 'great great', 'great great great', 'great great great great']:
    print( '%s -> %f' % (tweet, predict_tweet(tweet, freqs, theta)))

Expected Output:

I am happy -> 0.518580

I am bad -> 0.494339

this movie should have been great. -> 0.515331

great -> 0.515464

great great -> 0.530898

great great great -> 0.546273

great great great great -> 0.561561

In [ ]:
my_tweet = "@97sides CONGRATS :)" # give your custom input and check 
predict_tweet(my_tweet, freqs, theta)

Check performance using the test set
After training your model using the training set above, check how your model might perform on real, unseen data, by testing it against the test set.

**Instructions**: 

Implement test_logistic_regression
Given the test data and the weights of your trained model, calculate the accuracy of your logistic regression model.
Use your predict_tweet() function to make predictions on each tweet in the test set.

If the prediction is > 0.5, set the model's classification y_hat to 1, otherwise set the model's classification y_hat to 0.
A prediction is accurate when y_hat equals test_y. Sum up all the instances when they are equal and divide by m.

In [ ]:
def test_logistic_regression(test_x, test_y, freqs, theta):
    """
    This function should take (test_x, test_y, freqs, theta) as input and return accuracy as output, where
        test_x: a list of tweets
        test_y: (m, 1) vector with the corresponding labels for the list of tweets
        freqs: a dictionary with the frequency of each pair (or tuple)
        theta: weight vector of dimension (3, 1)
        accuracy: (# of tweets classified correctly) / (total # of tweets)
    """
    
    # the list for storing predictions
    y_hat = []
    
    for tweet in test_x:
        # get the label prediction for the tweet
        y_pred = predict_tweet(tweet, freqs, theta)  # call predict tweet function with proper parameters
        
        if y_pred > 0.5:
            # append 1.0 to the list
            y_hat.append(1)
            
            
        else:
            # append 0 to the list
            y_hat.append(0)
            

    # With the above implementation, y_hat is a list, but test_y is (m,1) array
    # convert both to one-dimensional arrays in order to compare them using the '==' operator
    accuracy =np.sum((y_hat== np.squeeze(test_y)).astype(int))/len(test_x)     # use squeeze function of numpy array on test_y and use appropiate expression.
    
    return accuracy

In [ ]:
#Expected output should match to your output. 
tmp_accuracy = test_logistic_regression(test_x, test_y, freqs, theta)
print(f"Logistic regression model's accuracy = {tmp_accuracy:.4f}")

Expected Output:
0.9950
Pretty good!

##Task 5: Predict with your own tweet - (5 points)¶

In [ ]:
#Take a tweet and predict whether it is Positive Sentiment ot Negative Sentiment
my_tweet ="I need to stop looking at old soccer pictures :(" #string containing the tweet
print(process_tweet(my_tweet))
y_hat = predict_tweet(my_tweet, freqs, theta)  # use appropriate function call to get prediction
print(y_hat)
if y_hat > 0.5:
    print('Positive sentiment')
else: 
    print('Negative sentiment')

##**PART-C**
¶
##**Theory:** (10 points)
Why is it generally preferable to use a Logistic Regression classifier rather than a classical Perceptron (i.e., a single layer of threshold logic units trained using the Perceptron training algorithm)? How can you tweak a Perceptron to make it equivalent to a Logistic Regression classifier? - 5 points

How many neurons do you need in the output layer if you want to classify email into spam or ham? What activation function should you use in the output layer? If instead you want to tackle MNIST, how many neurons do you need in the out‐ put layer, using what activation function? - 5 points

1)
 The logistic regression classifier is able to converge on non-linear data and outputs class probabilities. However, a classical perceptron will only converge if the data is linearly seperable.Also, it cannot compute class probabilities.

2)

Since it is a binary classification problem, you only need one neuron in the output layer. As for the activation function You'd have been using  the sigmoid in the output layer.

For the MNIST problem you would need 10 output neurons in the final layer, one for each digit. The  activation function in this case would be softmax.

